In [35]:
from torch.cuda import empty_cache as empty_cuda_cache, is_available as is_cuda_available
from torch.mps import empty_cache as empty_mps_cache
from torch.backends.mps import is_available as is_mps_available
from gc import collect

def reset_model(model, has_model: bool = True):
    if is_cuda_available(): empty_cuda_cache()
    elif is_mps_available(): empty_mps_cache()
    
    if has_model: del model

    collect()
    globals().clear()

%reset -f

In [36]:
from IPython.display import clear_output
from ultralytics import checks

clear_output()
checks()

Ultralytics YOLOv8.1.47 🚀 Python-3.11.5 torch-2.2.2 CPU (Apple M3 Max)
Setup complete ✅ (16 CPUs, 64.0 GB RAM, 664.9/1858.2 GB disk)


## Inference with Model

In [37]:
from torch import device
from torch.cuda import is_available as is_cuda_available
from torch.backends.mps import is_available as is_mps_available

BACKEND = "mps" if is_mps_available() else "cuda" if is_cuda_available() else "cpu"
DEVICE = device(BACKEND)
print(f"Device: {DEVICE} ({BACKEND})")

Device: mps (mps)


In [39]:
from os import getcwd
from os.path import dirname
 
HOME = dirname(getcwd())
print(HOME)

/Users/kiran/Documents/workspace/Projects/algae-detection


In [40]:
from IPython.display import Image, display

def output_img(path, height=600):
    display(Image(filename=path, width=height, height=height))

## Get custom dataset

In [13]:
from roboflow import Roboflow

rf = Roboflow(api_key="OZYVRHRpfy8DaxBPca6s")
project = rf.workspace("capstone2algae").project("algae-detection-1opyx")
VERSION = 9
version = project.version(VERSION)
#dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


## Custom Training and Validation

In [41]:
%cd {HOME}

yolov8_weights= f"{HOME}/weights/yolov8x.pt"
confidence = 0.25
epochs = 3
img_size = 256

/Users/kiran/Documents/workspace/Projects/algae-detection


using dhist requires you to install the `pickleshare` library.


In [ ]:
from ultralytics import YOLO

ds_location = f"{HOME}/dataset/algae_ds"
model = YOLO(model=yolov8_weights)
model.train(data=f"{ds_location}/data.yaml", plots=True, epochs=epochs, imgsz=img_size, device=DEVICE)

In [ ]:
output_img(f'{HOME}/runs/detect/train/confusion_matrix.png', 600)

In [ ]:
output_img(f'{HOME}/runs/detect/train/results.png', 600)

In [ ]:
output_img(f'{HOME}/runs/detect/train/val_batch0_pred.jpg', 600)

## Inference with Custom Model

In [ ]:
#load model
model = project.version(VERSION).model

#choose random test set image
from os import listdir
from random import choice

test_set_loc = f"{ds_location}/test/images/"
random_test_image = choice(listdir(test_set_loc))
print(f"running inference on {random_test_image}")
model = YOLO(model=f"{HOME}/weights/yolov8x.pt")

if model: model.predict(f"{test_set_loc}{random_test_image}", imgsz=img_size, conf=confidence, save=True, device=DEVICE)

In [ ]:
from glob import glob

for image_path in glob(f'{HOME}/runs/detect/predict/*.jpg'):
      output_img(image_path)
      print("\n")
#reset_model(model)

## Deploy model on Roboflow

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv8 weights.

To upload model weights, add the following code to the “Inference with Custom Model” section in the aforementioned notebook:

In [ ]:
#project.version(VERSION).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train")

version.deploy("yolov8", "weights", "best.pt")